In [ ]:
!pip install image-classifiers==1.0.0b1
!pip install keras_applications --no-deps
!pip install tensorflow==2.10

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
print(tf.__version__)

2.10.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Prepare CELEBA dataset

In [ ]:
gcs_base_dir = "gs://celeb_a_dataset/"
celeb_a_builder = tfds.builder("celeb_a", data_dir=gcs_base_dir, version='2.0.0')

celeb_a_builder.download_and_prepare()

#### Inspect CELEBA dataset element
- Each example is a dict of `{'attributes': {}, 'image': Tensor(shape=218, 178, 3), 'landmarks': {}}`
- 'attributes' is a dict of 40 features, each being a boolean Tensor of True/False
- We need 'attributes' and 'image' elements
- For 'attributes', we need to transform each feature from boolean to int 0/1
- For 'image', we need to resize it to specific format of our model. We will be using ResNet34 architecture (shown later), so we meed to transform the image to (224, 224, 3)

In [ ]:
# inspect celeba dataset element
celeb_a_builder.as_dataset(split='train')

<PrefetchDataset element_spec={'attributes': {'5_o_Clock_Shadow': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Arched_Eyebrows': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Attractive': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Bags_Under_Eyes': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Bald': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Bangs': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Big_Lips': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Big_Nose': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Black_Hair': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Blond_Hair': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Blurry': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Brown_Hair': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Bushy_Eyebrows': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Chubby': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Double_Chin': TensorSpec(shape=(), dtype=tf.bool, name=None), 'Eyeglasses': TensorSpec(s

In [ ]:
ATTR_KEY = "attributes"
IMAGE_KEY = "image"
IMAGE_SIZE = 224

In [ ]:
def preprocess_input_dict(feat_dict):
  # get image and attribute dict from the feature dictionary
  image = feat_dict[IMAGE_KEY]
  attr = feat_dict[ATTR_KEY]

  # cast each feature from boolean to float
  for k, v in attr.items():
    value = tf.cast(v, tf.float32)
    feat_dict[ATTR_KEY][k] = value

  # Resize and normalize image
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
  image /= 255.0

  feat_dict[IMAGE_KEY] = image

  return feat_dict

get_image_and_attr = lambda feat_dict: (feat_dict[IMAGE_KEY], feat_dict[ATTR_KEY])

In [ ]:
# prepare train/val/test dataset using preprocessing function defined above
train_ds = celeb_a_builder.as_dataset(split='train').shuffle(1024).map(preprocess_input_dict).map(get_image_and_attr)
val_ds = celeb_a_builder.as_dataset(split='validation').shuffle(1024).map(preprocess_input_dict).map(get_image_and_attr)
test_ds = celeb_a_builder.as_dataset(split='test').shuffle(1024).map(preprocess_input_dict).map(get_image_and_attr)

### Create model to train CELEBA

In [ ]:
def create_model(base_model, last_output):
  ''' 
  create a binary classifier for each face attribute in CELEBA, adding on top of ResNet34

  Args:
        base_model: ResNet34 model
        last_output: output from base ResNet34 model, which we will add classification layer on top
    
  Returns:
        new model used to train CELEBA
  '''
  Clock_Shadow_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='5_o_Clock_Shadow')(last_output)
  Arched_Eyebrows_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Arched_Eyebrows')(last_output)
  Attractive_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Attractive')(last_output)
  Bags_Under_Eyes_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Bags_Under_Eyes')(last_output)
  Bald_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Bald')(last_output)
  Bangs_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Bangs')(last_output)
  Big_Lips_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Big_Lips')(last_output)
  Big_Nose_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Big_Nose')(last_output)
  Black_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Black_Hair')(last_output)
  Blond_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Blond_Hair')(last_output)
  Blurry_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Blurry')(last_output)
  Brown_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Brown_Hair')(last_output)
  Bushy_Eyebrows_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Bushy_Eyebrows')(last_output)
  Chubby_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Chubby')(last_output)
  Double_Chin_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Double_Chin')(last_output)
  Eyeglasses_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Eyeglasses')(last_output)
  Goatee_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Goatee')(last_output)
  Gray_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Gray_Hair')(last_output)
  Heavy_Makeup_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Heavy_Makeup')(last_output)
  High_Cheekbones_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='High_Cheekbones')(last_output)
  Male_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Male')(last_output)
  Mouth_Slightly_Open_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Mouth_Slightly_Open')(last_output)
  Mustache_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Mustache')(last_output)
  Narrow_Eyes_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Narrow_Eyes')(last_output)
  No_Beard_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='No_Beard')(last_output)
  Oval_Face_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Oval_Face')(last_output)
  Pale_Skin_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Pale_Skin')(last_output)
  Pointy_Nose_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Pointy_Nose')(last_output)
  Receding_Hairline_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Receding_Hairline')(last_output)
  Rosy_Cheeks_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Rosy_Cheeks')(last_output)
  Sideburns_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Sideburns')(last_output)
  Smiling_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Smiling')(last_output)
  Straight_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Straight_Hair')(last_output)
  Wavy_Hair_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wavy_Hair')(last_output)
  Wearing_Earrings_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Earrings')(last_output)
  Wearing_Hat_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Hat')(last_output)
  Wearing_Lipstick_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Lipstick')(last_output)
  Wearing_Necklace_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Necklace')(last_output)
  Wearing_Necktie_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Necktie')(last_output)
  Young_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='Young')(last_output)

  all_outputs = [Clock_Shadow_branch,
                Arched_Eyebrows_branch,
                Attractive_branch,
                Bags_Under_Eyes_branch,
                Bald_branch,
                Bangs_branch,
                Big_Lips_branch,
                Big_Nose_branch,
                Black_Hair_branch,
                Blond_Hair_branch,
                Blurry_branch,
                Brown_Hair_branch,
                Bushy_Eyebrows_branch,
                Chubby_branch,
                Double_Chin_branch,
                Eyeglasses_branch,
                Goatee_branch,
                Gray_Hair_branch,
                Heavy_Makeup_branch,
                High_Cheekbones_branch,
                Male_branch,
                Mouth_Slightly_Open_branch,
                Mustache_branch,
                Narrow_Eyes_branch,
                No_Beard_branch,
                Oval_Face_branch,
                Pale_Skin_branch,
                Pointy_Nose_branch,
                Receding_Hairline_branch,
                Rosy_Cheeks_branch,
                Sideburns_branch,
                Smiling_branch,
                Straight_Hair_branch,
                Wavy_Hair_branch,
                Wearing_Earrings_branch,
                Wearing_Hat_branch,
                Wearing_Lipstick_branch,
                Wearing_Necklace_branch,
                Wearing_Necktie_branch,
                Young_branch]
  model = tf.keras.Model(inputs = base_model.input,
                         outputs = all_outputs)

  return model

In [ ]:
from classification_models.keras import Classifiers

# get ResNet model trained on ImageNet data
ResNet34, preprocess_input = Classifiers.get('resnet34')
base_model = ResNet34((224, 224, 3), weights='imagenet')

# get the output layer we want for training for CELEBA, which we will add classification layer on top
last_output = base_model.get_layer('pool1').output

# create model used to train CELEBA
model = create_model(base_model, last_output)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics='accuracy',)
model.summary()

87578640/87578640 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 224, 224, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)            

In [ ]:
hist = model.fit(
    train_ds.batch(64),
    validation_data = val_ds.batch(64),
    epochs = 20,
    callbacks = [EarlyStopping(monitor = 'val_loss', patience=3)]
)

Epoch 1/20
2544/2544 [==============================] - 330s 119ms/step - loss: 8.3836 - 5_o_Clock_Shadow_loss: 0.1620 - Arched_Eyebrows_loss: 0.3479 - Attractive_loss: 0.3936 - Bags_Under_Eyes_loss: 0.3447 - Bald_loss: 0.0331 - Bangs_loss: 0.1193 - Big_Lips_loss: 0.4753 - Big_Nose_loss: 0.3660 - Black_Hair_loss: 0.2514 - Blond_Hair_loss: 0.1313 - Blurry_loss: 0.1050 - Brown_Hair_loss: 0.3083 - Bushy_Eyebrows_loss: 0.2209 - Chubby_loss: 0.1241 - Double_Chin_loss: 0.1019 - Eyeglasses_loss: 0.0240 - Goatee_loss: 0.0847 - Gray_Hair_loss: 0.0621 - Heavy_Makeup_loss: 0.2034 - High_Cheekbones_loss: 0.3075 - Male_loss: 0.0738 - Mouth_Slightly_Open_loss: 0.1837 - Mustache_loss: 0.0826 - Narrow_Eyes_loss: 0.2631 - No_Beard_loss: 0.1232 - Oval_Face_loss: 0.5123 - Pale_Skin_loss: 0.1000 - Pointy_Nose_loss: 0.4882 - Receding_Hairline_loss: 0.1597 - Rosy_Cheeks_loss: 0.1299 - Sideburns_loss: 0.0725 - Smiling_loss: 0.2012 - Straight_Hair_loss: 0.3648 - Wavy_Hair_loss: 0.3550 - Wearing_Earrings_loss:

In [ ]:
def get_overall_accuracy(model_hist):
  '''
  Each feature's accuracy over 6 epochs is stored in a list from model_hist.history[feature_name], we want to compute all features' mean accuracy over 6 epochs
  To do so, we append each feature's training accuracy to acc list, validation accuracy to val list
  list will be like: acc = [[0.2, 0.3, 0.5, 0.6, 0.6, 0.7], # training accuracy for feature 1 over 6 epochs
                            [0.3, 0.4, 0.5, 0.6, 0.6, 0.7], # training accuracy for feature 2 over 6 epochs
                            ...]
  We then take average for each column, which will give us mean accuracy of all features in each epoch
  '''
  val_acc = []
  acc = []
  for k, v in model_hist.history.items():
    if k.startswith('val') and 'accuracy' in k:
      val_acc.append(v)
    elif 'accuracy' in k:
      acc.append(v)
  return np.mean(acc, axis=0), np.mean(val_acc, axis=0)

In [ ]:
acc, val_acc = get_overall_accuracy(hist)
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(acc)+1)

In [ ]:
acc

array([0.90780011, 0.917079  , 0.92154297, 0.92695767, 0.93426108,
       0.94311621])

In [ ]:
loss

[8.383584022521973,
 7.532598972320557,
 7.1286163330078125,
 6.66059684753418,
 6.045740127563477,
 5.297713756561279]

In [ ]:
val_acc

array([0.90400287, 0.91252831, 0.91331857, 0.91356395, 0.90941008,
       0.90671088])

In [ ]:
val_loss

[8.96402359008789,
 8.017656326293945,
 7.921825408935547,
 7.981600284576416,
 8.684476852416992,
 9.954047203063965]

### Evaluate trained model on test data

In [ ]:
metrics = model.evaluate(test_ds.batch(64), return_dict = True)

312/312 [==============================] - 51s 155ms/step - loss: 10.4267 - 5_o_Clock_Shadow_loss: 0.1543 - Arched_Eyebrows_loss: 0.4380 - Attractive_loss: 0.4120 - Bags_Under_Eyes_loss: 0.3851 - Bald_loss: 0.0300 - Bangs_loss: 0.1239 - Big_Lips_loss: 0.8930 - Big_Nose_loss: 0.4048 - Black_Hair_loss: 0.7167 - Blond_Hair_loss: 0.1340 - Blurry_loss: 0.1186 - Brown_Hair_loss: 0.3825 - Bushy_Eyebrows_loss: 0.2315 - Chubby_loss: 0.1326 - Double_Chin_loss: 0.1082 - Eyeglasses_loss: 0.0195 - Goatee_loss: 0.0678 - Gray_Hair_loss: 0.0647 - Heavy_Makeup_loss: 0.2373 - High_Cheekbones_loss: 0.3286 - Male_loss: 0.0643 - Mouth_Slightly_Open_loss: 0.2089 - Mustache_loss: 0.0858 - Narrow_Eyes_loss: 0.4400 - No_Beard_loss: 0.1184 - Oval_Face_loss: 0.5984 - Pale_Skin_loss: 0.1068 - Pointy_Nose_loss: 0.5906 - Receding_Hairline_loss: 0.1902 - Rosy_Cheeks_loss: 0.1686 - Sideburns_loss: 0.0664 - Smiling_loss: 0.2104 - Straight_Hair_loss: 0.4339 - Wavy_Hair_loss: 0.4104 - Wearing_Earrings_loss: 0.3071 - Wea

In [ ]:
print('total loss', metrics['loss'])

total loss 10.426733016967773


In [ ]:
test_acc = [v for k, v in metrics.items() if 'accuracy' in k]
print('mean accuracy over all features:', sum(test_acc)/40)

mean accuracy over all features: 0.9028504148125649


In [ ]:
print('mean loss over all features', metrics['loss']/40)

mean loss over all features 0.2606683254241943


### Save and load of trained model

In [ ]:
model.save('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

In [ ]:
model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 224, 224, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']   